# Multilabel BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
PREFIX = "voting_but" #"voting_so_automl"
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np
from collections import Counter

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data_file = f"../data/interim/{PREFIX}_withprompt.ndjson"

with open(data_file) as i:
    data = ndjson.load(i)
    
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)
    
labels = Counter()
for item in data:
    labels.update(item["label"])
print(labels)

I0327 11:00:27.577577 139890200905536 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0327 11:00:28.652792 139890200905536 file_utils.py:57] TensorFlow version 2.1.0 available.
I0327 11:00:29.296913 139890200905536 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Counter({'Personal Opinion to Fact': 557, "Outside of Article's Scope": 74, 'Governments Can Abuse It/Dictatorships': 52, 'Unintelligible Response': 25, 'Negative Effects on Voter': 20, 'Misuse of Conjunction': 16, 'Answered Wrong Stem': 15, 'Violation of Personal Liberty': 14, 'Reference': 14, 'Plagiarism': 13, 'Not All Governments Enforce It': 10, 'North Korea/Dictatorships': 10, 'Consequences/Bolivia': 10, 'Clarify/Identify/Specify': 10, 'Evidences Needs More Context': 10})


Read synthetic data, when available.

In [3]:
from collections import defaultdict

synth_files = glob.glob(f"../data/interim/{PREFIX}_withprompt_*.ndjson")

synth_data = []
for synth_file in synth_files:
    with open(synth_file) as i:
        synth_data.extend(ndjson.load(i))

preprocessed_synth_data = preprocess(synth_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
        
synth_map = defaultdict(list)
for item, preprocessed_item in zip(synth_data, preprocessed_synth_data):
    synth_map[item["source_text"]].append(preprocessed_item)

synth_map = {}

I0327 11:00:29.973740 139890200905536 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Model

We load the pretrained model and put it on a GPU if one is available. We also put the model in "training" mode, so that we can correctly update its internal parameters on the basis of our data sets.

In [4]:
def evaluate_output(all_correct, all_predicted):
    correct = 0
    at_least_one = 0
    fp, fn, tp, tn = 0, 0, 0, 0
    for c, p in zip(all_correct, all_predicted):
        if sum(c == p) == len(c):
            correct +=1

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                at_least_one += 1
                break

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                tp += 1
                same = 1
            elif pi == 1 and ci == 0:
                fp += 1
            elif pi == 0 and ci == 1:
                fn += 1
            else:
                tn += 1
                same =1

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print("Data size:", len(all_predicted))
    print("P:", tp, "/", tp+fp, "=", precision)
    print("R:", tp, "/", tp+fn, "=", recall)
    print("F:", 2*precision*recall/(precision+recall))
    print("A:", correct/len(all_correct))
    print("AL1:", at_least_one/len(all_correct))

In [5]:
import torch

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_multilabel_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/6*5)
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]
    
    print("Train size:", len(train_data))
    
    synth_data = []
    for item in train_data:
        synth_data.extend(synth_map.get(item.text, []))
                
    train_dataloader = get_data_loader(np.concatenate((train_data, synth_data)), BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    print("Final train dataloader length:", len(train_dataloader))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    evaluate_output(test_correct, test_predicted)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)

Train size: 543
Final train dataloader length: 34


I0327 11:00:30.517525 139890200905536 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 11:00:30.519131 139890200905536 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Loss history: []
Dev loss: 0.42688906618527006


Epoch:   5%|▌         | 1/20 [00:08<02:32,  8.01s/it]


Loss history: [0.42688906618527006]
Dev loss: 0.19715909659862518


...



Epoch:  60%|██████    | 12/20 [01:34<01:01,  7.70s/it]


Loss history: [0.4018698973315103, 0.20610932792936051, 0.17019817658833095, 0.1632333674601146, 0.158241582768304, 0.15739339802946364, 0.15474334252732141, 0.15025439326252257, 0.15668501066310064, 0.16096518295151846, 0.16030167468956538]
Dev loss: 0.15496908447572164



Loss history: [0.4018698973315103, 0.20610932792936051, 0.17019817658833095, 0.1632333674601146, 0.158241582768304, 0.15739339802946364, 0.15474334252732141, 0.15025439326252257, 0.15668501066310064, 0.16096518295151846, 0.16030167468956538, 0.15496908447572164]
Dev loss: 0.1591213888355664
No improvement on development set. Finish training.
Loading model from /tmp/model.bin


I0327 11:12:12.929546 139890200905536 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 11:12:12.930419 139890200905536 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Data size: 163
P: 115 / 134 = 0.8582089552238806
R: 115 / 170 = 0.6764705882352942
F: 0.7565789473684211
A: 0.7055214723926381
AL1: 0.7055214723926381


## Evaluation

In [6]:
evaluate_output(all_correct, all_predicted)

Data size: 816
P: 553 / 633 = 0.8736176935229067
R: 553 / 850 = 0.6505882352941177
F: 0.7457855697909643
A: 0.6715686274509803
AL1: 0.6776960784313726


In [7]:
scores = {}
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    correct_labels = [idx2label[i] for i, l in enumerate(correct) if l == 1]
    predicted_labels = [idx2label[i] for i, l in enumerate(predicted) if l == 1]
    print("{}#{}#{}".format(item.text, ";".join(correct_labels), ";".join(predicted_labels)))    
        
    for label in predicted_labels + correct_labels:
        if label not in scores:
            scores[label] = {"tp": 0, "fp": 0, "fn": 0, "support": 0}
    
    for label in predicted_labels:
        if label in correct_labels:
            scores[label]["tp"] += 1
        else:
            scores[label]["fp"] += 1

    for label in correct_labels:
        scores[label]["support"] += 1
        if label not in predicted_labels:
            scores[label]["fn"] += 1
            
for label in scores:
    lp = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fp"]) if scores[label]["tp"] + scores[label]["fp"] > 0 else 0
    lr = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fn"]) if scores[label]["tp"] + scores[label]["fn"] > 0 else 0
    lf = 2*lp*lr/(lp+lr) if lp+lr > 0 else 0
    
    print(label, lp, lr, lf, scores[label]["support"])

governments often misuse this law#Governments Can Abuse It/Dictatorships#
it is something that could easily be corrupted and abused by certain governments.#Governments Can Abuse It/Dictatorships#Governments Can Abuse It/Dictatorships
, sometimes the government abuses compulsory voting.#Governments Can Abuse It/Dictatorships#
dictator-like governments can abuse it to support certain political groups.#Governments Can Abuse It/Dictatorships#Governments Can Abuse It/Dictatorships
governments can also sometimes misuse it.#Governments Can Abuse It/Dictatorships#Governments Can Abuse It/Dictatorships
governments can sometimes misuse it#Governments Can Abuse It/Dictatorships#Governments Can Abuse It/Dictatorships
sometimes it can be misused by the government.#Governments Can Abuse It/Dictatorships#Governments Can Abuse It/Dictatorships
this may cause hardships for voters who are not able to make it for good reasons#Negative Effects on Voter#
voters may be given harsh consequences for failure t

In [8]:
print(len([p for p in all_predicted if sum(p) == 0]), "/", len(all_predicted))

183 / 816
